In [ ]:
# IMPORT YOUR LIBRARIES HERE
import random

import matplotlib as mp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
import prettytable as pt
import seaborn as sns
from sklearn.feature_selection import RFE
from sklearn.linear_model import (LinearRegression, LogisticRegression,
                                  LogisticRegressionCV, Ridge, RidgeCV)
from sklearn.metrics import (confusion_matrix, f1_score, mean_absolute_error,
                             mean_squared_error, precision_score, r2_score,
                             recall_score)
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import (LabelEncoder, MinMaxScaler, OneHotEncoder,
                                   PolynomialFeatures, StandardScaler)
from sklearn.tree import plot_tree

In [ ]:
# Ignoring future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


# Assignment 1

Welcome to the assignment!

You will have to implement regression and classification algorithms, applying these methods to the topics of agriculture, food, water, and health. More precisely, you will try to:

- predict crop yields using data on weather and fertilizer use;
- predict the potability of water using data on the mineral and micro-organisms content of water.

Once you are done you have to submit your notebook here:
[https://moodle.epfl.ch/mod/assign/view.php?id=1244180](https://moodle.epfl.ch/mod/assign/view.php?id=1244180)

If there is need for further clarifications on the questions, after the assignment is released, we will update this file, so make sure you check the git repository for updates.

Good luck!


## Linear regression: predicting crop yields

In 2020, between 720 million and 811 million persons worldwide were suffering from hunger (see [SDG Goal 2](https://www.un.org/sustainabledevelopment/hunger/) Zero Hunger). Given the ongoing growth of the world population, it is imperative to comprehend crop yield at a global level in order to tackle food security issues and mitigate the effects of climate change.

The Agricultural yield depends on weather conditions (rain, temperature, etc) and fertilizers use. Having precise information regarding the historical crop yield is critical for making informed decisions regarding agricultural risk management and future projections.

Some E4S publications on the topic of food:

- [Threats to Nitrogen Fertilizer, Opportunities to Cultivate Sustainable Practices?](https://e4s.center/resources/reports/threats-to-nitrogen-fertilizer-opportunities-to-cultivate-sustainable-practices/)
- [True cost of food as a lever to transform the Swiss food system](https://e4s.center/resources/reports/true-cost-of-food-as-a-lever-to-transform-the-swiss-food-system/)

<img src='https://upload.wikimedia.org/wikipedia/commons/thumb/b/b7/Sustainable_Development_Goal_02ZeroHunger.svg/800px-Sustainable_Development_Goal_02ZeroHunger.svg.png' width='200'>

We will use data obtained from the [FAO](http://www.fao.org/home/en/) (Food and Agriculture Organization) and [World Data Bank](https://data.worldbank.org/), and gathered in the [Crop Yield Prediction Dataset](https://www.kaggle.com/datasets/patelris/crop-yield-prediction-dataset).

Our goal is to predict the crop yields using the temperature, rain fall, and type of crops.


### Question 1: Load and Discover the dataset


- Load the data in a dataframe. The url link is provided below. Display the first 10 observations and the types of data **1 point**


In [ ]:
url_yield = 'https://raw.githubusercontent.com/michalis0/MGT-502-Data-Science-and-Machine-Learning/main/data/yield_df.csv'

# local_path = '../data/yield_df.csv'
yield_df = pd.read_csv(url_yield)

I will start by renaming the columns for improved readability.

In [ ]:
yield_df = yield_df.rename(index=str, columns={'year': 'Year', 'hg/ha_yield': 'Yield', 'avg_temp': 'Temperature',
                           'pesticides_tonnes': 'Pesticides', 'average_rain_fall_mm_per_year': 'Avg. Rain fall'})
display(yield_df.head(10))
display(yield_df.dtypes)


- Print the list of countries ('Area') and years available in the dataset **1 point**


In [ ]:
# Function that returns all the unique (if set) items in the given column

def get_col_items_as_list(df, col_name, unique) -> list:
    '''
    Function that returns all the columns names in a list of a given df
    The parameter unique is used to check for unique items in the column
    '''
    if unique:
        return df[col_name].unique().tolist()
    else:
        return df[col_name].value.tolist()


In [ ]:
countries = get_col_items_as_list(yield_df, 'Area', True)
years = get_col_items_as_list(yield_df, 'Year', True)

print(f'The countries in the the dataframe are:\n {countries}')
print(f'The years available in the the dataframe are:\n {years}')


- Print the list of 'Item' in the dataset. You should obtain a list of 10 crops, which are among the most consumed in the world **1 point**


In [ ]:
items = get_col_items_as_list(yield_df, 'Item', True)
print(
    f'The most consumed crops are: \n{items}\nAnd the number of crops are: {len(items)}')


- Display summary statistics for the columns: 'hg/ha_yield', 'average_rain_fall_mm_per_year', 'pesticides_tonnes','avg_temp'. How many observations do we have? **1 point**

_Hint:_ You can extract the columns 'hg/ha_yield', 'average_rain_fall_mm_per_year', 'pesticides_tonnes','avg_temp' in a new dataframe since we will reuse it in the following questions


In [ ]:
# Function to create a copy of a df with the given columns

def get_df_subset(df, cols) -> pd.DataFrame:
    return df[cols].copy()


In [ ]:
columns_to_extract = [
    'Yield', 'Avg. Rain fall', 'Pesticides', 'Temperature']
yield_subset = get_df_subset(yield_df, columns_to_extract)
display(yield_subset.describe())
print(
    f'We have {yield_subset.shape[0]} observations in our new subset dataframe')


- Display a heatmap of the correlation matrix between the columns: 'hg/ha_yield', 'average_rain_fall_mm_per_year', 'pesticides_tonnes','avg_temp' **1 point**


In [ ]:
def get_heatmap(df, title, color_label='', color='RdBu_r', vmin=-1, vmax=1, annot_size=15) -> plt.figure:
    '''
    Function to get a heatmap with either all columns or a subset of the columns
    Returns heatmpap.
    '''
    sns.set(font_scale=1.1)


    plt.figure(figsize=(2*df.shape[1], df.shape[1]))
    heatmap = sns.heatmap(
        df.round(2),
        cmap=color,
        annot=True,
        fmt='.4g',
        vmin=vmin, vmax=vmax,
        annot_kws={'size': annot_size},
        cbar_kws={'label': color_label, 'orientation': 'vertical'})

    plt.title(title)
    return heatmap


In [ ]:
heatmpap_q1 = get_heatmap(yield_subset.corr(), title='Correlation Matrix Crop Parameters', color_label='Correlation')

Here we can see that rainfall (mm/year) and Hg/Ha Yield has the lowest correlation of the pairs. The largest positive correlation is between rainfall (mm/year) and average temperature. The larges negative correlation is between average temperature and rainfall (mm/year).


- Create a boxplot of the columns: 'hg/ha_yield', 'average_rain_fall_mm_per_year', 'pesticides_tonnes', 'avg_temp' **1 point**


In [ ]:

def get_box_plot(data: pd.DataFrame, title='Box plot'):
    '''
    Function that creates a box plot on the given data
    Title can be changed
    '''
    fig, axs = plt.subplots(nrows=1, ncols=data.shape[1], figsize=(15, 5))

    # Plot each set of data in a different subplot
    for i in range(data.shape[1]):
        axs[i].boxplot(data.iloc[:, i], widths=0.6, patch_artist=True,
                       boxprops=dict(facecolor='lightblue', color='black'))
        axs[i].set_title(f'{data.iloc[:,i].name}')

    fig.text(0.04, 0.5, 'Observations', rotation='vertical')
    fig.suptitle(title)
    # Increase spacing between subplots
    fig.subplots_adjust(wspace=0.4)
    return fig


In [ ]:
box_plot_q1 = get_box_plot(yield_subset, 'Distribution of Crop parameters')

- Create a pairplot of the columns: 'hg/ha_yield', 'average_rain_fall_mm_per_year', 'pesticides_tonnes', 'avg_temp' **1 point**


In [ ]:
def plot_pairplot(data, title='', diag_color='red', hue=None, columns=None):
    ''' 
    Function to get pairplot, input: dataframe, title and diagonal color
    Returns plot
    '''
    if columns:
        data=data[columns]

    # Customize the diagonal plots to highlight the distribution and setting hue (if given) on given color
    plot = sns.pairplot(data, hue=hue, diag_kws={'color': diag_color})
    
    # Set the title of the plot
    plt.subplots_adjust(top=0.9)
    plot.fig.suptitle(title, fontsize=18)
    
    # Adjust the spacing between the subplots
    plot.fig.subplots_adjust(wspace=0.2, hspace=0.2)
    
    return plot

In [ ]:
pair_plot_q1 = plot_pairplot(yield_subset, title='Data distribution')

On the diagonal (with a slight red color) we can see the distribution. All of them are quite skewed either left or right except rainfall (mm/year) which is more equally distributed. 


- Feel free to pursue your exploration to better understand your dataset. Although not graded, this might help you better understanding the problem and answer the following questions.


First of all i just wanted to see have all of these paremeters have changed over time. Some of them might have increased a lot the last few years, and can then also give us some information about the parameters respective importance and relationship to later predict our crop yield. Additionally i would like to check whether i have some duplicates in my data, plot its main statistics, and if its a substantial amount it might bias our data. 

In [ ]:
# Getting duplicates
duplicates_to_show = yield_df.duplicated(keep=False)
duplicates = yield_df.duplicated()
duplicate_rows = duplicates[duplicates]

# Displaying duplicates
display(yield_df.loc[duplicates_to_show])
print('Number of duplicates:', duplicates.sum())

# Comparing the characteristics of the duplicated with the full data set and a dataset with removed duplicates
display('Duplicate dataset', yield_df.loc[duplicates].describe())
display('Original dataset', yield_df.describe())

yield_no_dup_df = yield_df.drop_duplicates()
display('No duplicates dataset', yield_no_dup_df.describe())


Our 'No Duplicate' df does not differ a lot, but when we will train a model later, we would put extra emphasis on the values that occur most frequently. We can see that the mean Pesticide $(~37077 > ~34783)$ usage got a bit lower. Because of this extra emphasis and weight to the observations we have duplicates of, i will remove them and continue working on the dataframe without duplicates. So, after discussing with Boris Thurm about these findings, we where a bit unsure how to interpret it, I tried looking up at the original data provider, and it might seem that these might not be duplicates but just approximate for different regions within a country, so I will continue with the assumption that there might be another parameter missing that differentiates these numbers. Thus i will continue with the original dataset.

So let's check the development of the Yield through time.

In [ ]:
# Creating a dataframe for items ,yield according to year
yield_crop_per_year = yield_df.groupby(['Item', 'Year'])[
    'Yield'].aggregate('sum').reset_index(name='Yield')


In [ ]:
def get_plotly_plot(df, x, y, title, color=None):
    '''
    Function that creates a plotly plot on given dataframe, x, y columns and title.
    Takes also a color that corresponds to a column, if not given defaults to None. 
    '''
    fig = px.line(df, x=x, y=y,
                  color=color, hover_name=y,  hover_data={x: ':.1f', y: ':.1f'})

    fig.update_layout(
        title=title,
        xaxis_title=x,
        yaxis_title=y,
        font=dict(
            family='Times New Roman',
            size=10,
            color='Black'
        )
    )
    return fig


In [ ]:
crops_through_time = get_plotly_plot(
    yield_crop_per_year, 'Year', 'Yield', 'Crop yield for each crop type over time', 'Item')
crops_through_time.show()


We can see that the yield is steadily increasing for all crop types (items).

In [ ]:
# Creating a dataframe for items ,yield according to year
yield_persticides_per_year = yield_df.groupby(
    ['Item', 'Year'])['Pesticides'].aggregate('sum').reset_index(name='Pesticides')


In [ ]:
pesticides_through_time = get_plotly_plot(
    yield_persticides_per_year, 'Year', 'Pesticides', 'Pesticides for each crop through time', 'Item')
pesticides_through_time.show()


The pesticide usage has heavily increased!

We could also see that the data was heavily skewed, i will try to do a log transformation on the most skewed columns (Yield and Pesticides), and see if that has any impact on our predictions later.

In [ ]:
yield_subset_norm = yield_subset.copy()
yield_subset_norm['Yield'] = np.log(yield_subset_norm['Yield'])
yield_subset_norm['Pesticides'] = np.log(yield_subset_norm['Pesticides'])
display(yield_subset_norm)
box_plot_q1_norm = get_box_plot(yield_subset_norm, 'Log normalized data distribution')
pair_plot_q1 = plot_pairplot(yield_subset_norm, 'Log normalized data distribution')


The two plots above clearly shows that the distribution is not as aggressive anymore. I will investigate whether the log transformed data performs better in the coming question.

### Question 2: Multivariate regression

We will try to predict the crop yields (column 'hg/ha_yield') using as features: 'Item', 'average_rain_fall_mm_per_year', 'pesticides_tonnes', 'avg_temp'

- Extract your features and outcome **1 point**


What we are trying to predict can be explained by the following equation:

$$Yield = w_0 + w_1 * Item + w_2 * Rainfall + w_3 * Pesticides + w_4 * Temperature + \epsilon_i$$
Where $w_i$ represents the different weights we assign to the given parameters.


In [ ]:
features = ['Item', 'Avg. Rain fall',
            'Pesticides', 'Temperature']

df_multi = get_df_subset(yield_df, features)


X = df_multi
y = yield_df['Yield']

# Keeping the log transformed data to check whether that can have an impact on the results later on
X_log = yield_subset_norm.copy()
X_log['Item'] = df_multi['Item'].copy()
X_log.drop(columns='Yield', inplace=True)
y_log = yield_subset_norm['Yield']

- Split between training and test set **1 point**

_Note_: Use as option: `test_size=0.2`, `random_state=42`, `shuffle=True`


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=True)


- Encode the column 'Item' using `LabelEncoder` **1 point**

_Note_: After training your encoder, you need to transform the values of both the training and test set


In [ ]:

print(X_train[['Item']])

# Extract the column of interest
Item = X_train[['Item']].values.ravel()
Item_test = X_test[['Item']].values.ravel()

# Define the encoder
le = LabelEncoder()

# Fit the encoder
le.fit(Item)

# Transform the train and the test set
X_train = X_train.assign(Item=le.transform(Item))
X_test = X_test.assign(Item=le.transform(Item_test))
print(X_train[['Item']])


Now we have transformed the categorical variables by encoding the different 'items'.


- Rescale your features using `MinMaxScaler` **1 point**


In [ ]:
def get_scaler(X_train:list, X_test:list, scaler):
    '''
    Function that returns scaled values for the input of X's and Y's, with the given scaler
    Returns a tuple with the scaled X,Y values
    '''
    scaler = scaler()
    scaler.fit(X_train)  # Fitting the scaler

    # Transform the train and the test set
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    return X_train, X_test

min_max_scaler = MinMaxScaler  # Instantiating the scaler
X_train, X_test = get_scaler(X_train, X_test, min_max_scaler)


- Build and train a multivariate linear regression model **1 point**


In [ ]:
# Instantiating the linear regression model
model = LinearRegression()

# Train model
model.fit(X_train, y_train)


- What is the $R^2$, mean absolute error, and mean square error on the training set? On the test set? What do you think? **1 point**


In [ ]:
def get_mae_mse_r2(x, y, model) -> tuple:
    '''
    Function that given a model calculates the MAE, MSE and R-squared rounded with 3 decimals
    '''
    # Model predictions
    pred = model.predict(x)

    mae = mean_absolute_error(y, pred).round(3)
    mse = mean_squared_error(y, pred).round(3)
    r2 = r2_score(y, pred).round(2)

    return mae, mse, r2


In [ ]:

def get_mae_mse_r2_df(X_train, y_train, X_test, y_test, model, column_name) -> pd.DataFrame:
    '''
    Function that returns a dataframe with the key statistical features (MAE, MSE and R-squared) on training and test set
    Uses the helper function get_mae_mse_r2()
    '''

    mae_train, mse_train, r2_train = get_mae_mse_r2(X_train, y_train, model)
    mae_test, mse_test, r2_test = get_mae_mse_r2(X_test, y_test, model)

    stat_tests_labels = ['MAE Train',
                  'MAE Test',
                  'MSE Train',
                  'MSE Test',
                  'R\u00b2 Train',
                  'R\u00b2 Test']

    df = pd.DataFrame([mae_train, mae_test, mse_train, mse_test, r2_train, r2_test],
                      index=stat_tests_labels,
                      columns=[column_name])
    
    # Changing the format and limiting to 2 decimals
    pd.options.display.float_format = '{:.3f}'.format

    return df


In [ ]:
multi_var_stats_df = get_mae_mse_r2_df(X_train, y_train, X_test, y_test, model, 'Multivariate regression')
display(multi_var_stats_df)


Here we can see the statistical performance of our values with the label encoder. As we know the label encoder gives each crop type its own value. Now we can check that the categorical values actual was transformed to.

In [ ]:
display(np.unique(le.transform(Item)))
display(np.unique(X_train.T[0]))
display(len(np.unique(X_train.T[0])) == np.unique(le.transform(Item)))

At least to me it's not intuitive that we will have all the different crop types in the same dimension. Here we can see that each is mapped in the range of [0,1] with a 0.11111111 interval. I'm just thinking out loud here, but how can the model build something useful on the fact that Soybean is 0.11111111 larger than for instance Sorghum. I would assume that having one column for each crop type would provide our model more predictive power.

So to conclude, overall the $R^2$ is very low, which means that our models has a bad fit. Our model has a $R^2$ close to $0$, which means that our model is nearly better to predict than taking the average of our observations. We can also see that the $R^2$ on the training set is in absolute terms $0.01$ lower than the test set. This difference is very low and can just be explained by randomness. Both the $MAE$ and $MSE$ is slightly bigger on the test set.

It seems counter-intuitive that the $R^2$ is bigger on the test set when the $MSE$ and $MAE$ is larger as well. The reason for this is that $TSS\text{ (total sum of squares)}$ is relatively smaller than the $RSS\text{ (residual sum of squares)}$ in the test set compared to the training set.


#### Question 2.1: Multivariate regression with log transformed data
It would be interesting to see how the log transformed data will perform in this matter. So the following code does exactly the same steps but with the log-transformed data.

In [ ]:
# 1. Splitting train and test set
X_train_log, X_test_log, y_train_log, y_test_log = train_test_split(
    X_log, y_log, test_size=0.2, random_state=42, shuffle=True)

# 2. Encoding the categorical data

Item = X_train_log[['Item']].values.ravel()
Item_test_log = X_test_log[['Item']].values.ravel()
le = LabelEncoder()
le.fit(Item)
X_train_log = X_train_log.assign(Item=le.transform(Item))
X_test_log = X_test_log.assign(Item=le.transform(Item_test_log))

# 3. Scaling data
X_train_log, X_test_log = get_scaler(X_train_log, X_test_log, min_max_scaler)

# 4. Building and train model
model_log = LinearRegression()
model_log.fit(X_train_log, y_train_log)

# 5. Testing model
multi_var_stats_df_log = get_mae_mse_r2_df(X_train_log, y_train_log, X_test_log, y_test_log, model_log, 1)

multi_var_comparison = multi_var_stats_df.copy()
multi_var_comparison.rename(columns={1: 'Absolute'}, inplace=True)
multi_var_comparison['Multivariate regression (Log transformed)'] = multi_var_stats_df_log
display(multi_var_comparison)
display(multi_var_comparison.dtypes)
 

We can see that the $R^2$ is slightly better for the log transformed data, but when some data is in log and other i absolute numbers, it's harder to interpret.

### Question 3: Polynomial features regression

We will try to improve the quality of our prediction using `PolynomialFeatures`.

- Write a function that is using as inputs the degree of polynomial features (an integer), the training and test set (for your features and outcome), and return the $R^2$, mean absolute error, and mean square error on the training and on the set of a polynomial feature regression **3 points**

_Hint:_ You do not need to include in your function the splitting, encoding and scaling since we will reuse the ones set created before (as before). Your function should transform your training and test set to integrate polynomial features, then build and train your model, before calculating the various error metrics.


In [ ]:
def get_poly_transform(X_train, X_test, degrees) -> tuple:
    '''
    Helper function to get polynomial transformation returns tuple with train and test set
    '''
    model_poly = PolynomialFeatures(degrees)
    X_train_poly = model_poly.fit_transform(X_train)
    X_test_poly = model_poly.transform(X_test)
    return X_train_poly, X_test_poly


In [ ]:

def get_poly_stats(degrees, X_train, y_train, X_test, y_test, additonal_title='') -> pd.DataFrame:
    '''
    Function that create model using get_poly_transform() and trains the model with the given data
    Returns a dataframe with  key statistical characteristics s.a.a MAE, MSE, R^2 using get_mae_mse_r2_df()
    '''

    X_train_poly, X_test_poly = get_poly_transform(
        X_train, X_test, degrees)

    # Setting up the model
    model_poly = LinearRegression(fit_intercept=False)

    # Fit
    model_poly.fit(X_train_poly, y_train)

    # Now we can reuse the previous made functions to get the MAE, MSE and R^2
    return get_mae_mse_r2_df(X_train_poly, y_train, X_test_poly, y_test, model_poly, f'Poly Degree: {degrees} {additonal_title}')


- What are the the $R^2$, mean absolute error, and mean square error on the training and on the set of a polynomial features regression with degree = 3? With degree = 7? **1 point**


In [ ]:
poly_stats_df_3 = get_poly_stats(3, X_train, y_train, X_test, y_test)
poly_stats_df_7 = get_poly_stats(7, X_train, y_train, X_test, y_test)

display(poly_stats_df_3)
display(poly_stats_df_7)


- Plot the evolution of the MSE on the training set for a polynomial feature regression model when the degree goes from 2 to 10. On the same figure, plot the MSE on the test set for a polynomial feature regression model, when the degree goes from 2 to 10. Which degree would you choose and why? **2 points**


In [ ]:
one_to_ten = np.linspace(1, 10, num=10, dtype='int')
df_stats_poly_one_ten = pd.DataFrame([])
df_stats_poly_one_ten_log = pd.DataFrame([])

# Looping from 1-10 to get the stats for each polynomial degree
for i in one_to_ten:
    df_stats_poly_one_ten[f'Polynomial degree: {i}'] = get_poly_stats(
        i, X_train, y_train, X_test, y_test)
    # Doing the same with the log transformed data to check if it makes a difference
    df_stats_poly_one_ten_log[f'Polynomial degree: {i} (Log)'] = get_poly_stats(
        i, X_train_log, y_train_log, X_test_log, y_test_log)

display(df_stats_poly_one_ten)


Now we have a dataframe with all the data we need to plot the development. I will create a plot for the $MSE$, to see more easily compare the log transformed data with the .


In [ ]:

def get_plot_with_traces(df: pd.DataFrame, y_list, title) -> go.Figure:
    '''
    Function that returns a graph with multiple traces, input is a dataframe and a list of the y-traces to be plotted along with title
    '''
    # Define the x and y values
    x = df.columns.values
    trace_list = []

    for y in y_list:
        trace = go.Scatter(
            x=x, y=y.values,
            mode='lines+markers',
            name=y.name,
            hovertemplate='Polynomial degree: %{x}<br>%{y}',
            marker=dict(size=8),
            line=dict(
                width=2,
            )
        )
        trace_list.append(trace)

    # Add axis labels and a title
    layout = go.Layout(
        title=title,
        xaxis=dict(title='Polynomial degrees'),
        yaxis=dict(title='R\u00b2'),
        legend=dict(x=1, y=-0.2),
        autosize=True
    )

    # Create a figure
    return go.Figure(data=trace_list, layout=layout)


In [ ]:


# Define the x and y values
y1 = df_stats_poly_one_ten.iloc[-4,1:]  # Getting the fourth last row : MSE train
y2 = df_stats_poly_one_ten.iloc[-3,1:]  # Getting the third last row : MSE test

fig = get_plot_with_traces(df_stats_poly_one_ten.iloc[:, 1:], y_list=[
#fig = get_plot_with_traces(df_stats_poly_one_ten, y_list=[
                           y1, y2], title='MSE for different polynomial degrees')
fig.update_layout(yaxis=dict(title='MSE'))
fig.show()


In [ ]:

# Define the x and y values
y_log_1 = df_stats_poly_one_ten_log.iloc[-4,1:]  # Getting the fourth last row : MSE train
y_log_2 = df_stats_poly_one_ten_log.iloc[-3,1:]  # Getting the third last row : MSE test

fig_log = get_plot_with_traces(df_stats_poly_one_ten_log.iloc[:, 1:], y_list=[
                           y_log_1, y_log_2], title='MSE for different polynomial degrees (log transformed)')
fig_log.update_layout(yaxis=dict(title='MSE'))
fig_log.show()


Here we can clearly see that the $R^2$ increases a lot for every iteration towards a polynomial degree of 6. After 6, it diverges as our model is getting too 'specialized' and lacking generalization. But when our features get larger than the number of observation there's an increasing chance of overfitting. 

Let's say that:
$$\text{d = degree, m = features, n=observations, and k = "new" number of features}$$
We know that the amount of polynomial features (k) are given as:
$$ k =\begin{pmatrix}d+m\\d\end{pmatrix} = \frac{(d + m)!}{(d! * m!)}.$$



Scikit uses the L-FBGS algorithm with time complexity  $O(k*m)$. Where $m$ is a number describing the memory (typically 5-30), but the essence is that the running time is highly dependent on number of features ($k$). 
I would therefor choose 6, which matches the "elbow" of $MSE$ ('MSE for different polynomial degrees') of the graph plotted earlier. With $d=6$ and $m=4$ we get to following amount of new features ($k$), which should be computational "feasible".
$$\text{Polynomial Degree} = 6$$
$$ k =\begin{pmatrix}6+4\\6\end{pmatrix} = \frac{(6 + 4)!}{(6! * 4!)} = 210$$


Sources:
- http://www.people.vcu.edu/~ysong3/lecture8.pdf
- https://aip.scitation.org/doi/pdf/10.1063/1.4995124
- https://en.wikipedia.org/wiki/Limited-memory_BFGS



For curiosity sake, and since I have made  a generic function to plot several traces, we can easily plot the $R^2$ of both our log and "normal" data.

In [ ]:
# Rename for readability in graph

y1 = df_stats_poly_one_ten.iloc[-2]  # Getting the second last row : R^2 train
y2 = df_stats_poly_one_ten.iloc[-1]  # Getting the  last row : R^2 test
y3 = df_stats_poly_one_ten_log.iloc[-1]
y4 = df_stats_poly_one_ten_log.iloc[-2]

df_stats_poly_one_ten_log.rename(
    index={'R\u00b2 Train': 'R\u00b2 Train (Log)'}, inplace=True)
df_stats_poly_one_ten_log.rename(
    index={'R\u00b2 Test': 'R\u00b2 Test (Log)'}, inplace=True)

# Create a list to feed the plot function for the traces i want to display
y_values_to_be_traced = [y1, y2, y3, y4]

r2_plot_comparison = get_plot_with_traces(df_stats_poly_one_ten, y_list=y_values_to_be_traced,
                                          title='R\u00b2 for different polynomial degrees (including log transformed)')
r2_plot_comparison.show()


From this graph we can see that the log normalized data doesn't really outperform the absolute values even though it performs slightly better except at polynomial degree 4, where they are approximately the same. For simplicity, and easier interpretation i will remain with the original data format.

### Question 4: Ridge and cross-validation

- Build, train, and evaluate a polynomial features regression model, with Ridge regularization, and cross validation. For number of degree, select the one that you picked before. How does your new model compares to your previous one? **3 points**


In [ ]:

# Use helper function to get poly transformation for the chosen polynomial degrees
X_train_poly_ridge, X_test_poly_ridge = get_poly_transform(X_train, X_test, 6)

# Set up the model
ridge_model = Ridge(alpha=1, fit_intercept=False, max_iter=1000)
ridge_model_zero = Ridge(alpha=0, fit_intercept=False, max_iter=1000)

# Use fit
ridge_model.fit(X_train_poly_ridge, y_train)
ridge_model_zero.fit(X_train_poly_ridge, y_train)

# Getting the key stats df
ridge_stats = get_mae_mse_r2_df(
    X_train_poly_ridge, y_train, X_test_poly_ridge, y_test, ridge_model, 'Ridge Model (Alpha=1)')

ridge_stats_zero = get_mae_mse_r2_df(
    X_train_poly_ridge, y_train, X_test_poly_ridge, y_test, ridge_model_zero, 'Ridge Model (Alpha=0)')

# Displaying with 2 decimals in string format for readability
display(ridge_stats)
display(ridge_stats_zero)


The results where not very impressive by the ridge regularization, but ridge punishes the test data quite substantially since the regularization parameter $\alpha$ has a large impact on $MAE$ and $MSE$ (in the test data). When `alpha = 0`, the objective is equivalent to ordinary least
squares. Thus when $\alpha$ is approaching $0$, our $R^2$ approaches $0.77$ which is what we got with our 7 degree polynomial prediction.

Now we can compare these statistics with the previous created df called `df_stats_poly_one_ten` where we have the performance metrics for $\text{Polynomial Degree}: 1-10$.


In [ ]:
poly_and_ridge_comparison_df = df_stats_poly_one_ten.copy()
poly_and_ridge_comparison_df[ridge_stats.columns[0]] = ridge_stats
poly_and_ridge_comparison_df[ridge_stats_zero.columns[0]] = ridge_stats_zero
display(poly_and_ridge_comparison_df)


As we saw above, the results have not improved, in addition to the way $\alpha$ punishes it it might be that the way we transformed our categorical data with LabelEncoding is not the optimal.


Now let's use K-fold cross validation and see if our score improves. Just to showcase the best alpha, we can use the parameter `alphas` for the cross-validation to find the best alpha. The parameter has to be $>0$, so we can start with something really small such as $1*10^{-10}$ and create a evenly spaced interval up to $100$. We can also report the best $\alpha$ to verify our findings.


In [ ]:
# Set up the model
ridge_cv_model = RidgeCV(alphas=np.linspace(
    1e-10, 100, 20), cv=5, fit_intercept=False)

# Use fit and the previous created X-values, the y stays the same
ridge_cv_model.fit(X_train_poly_ridge, y_train)

ridge_cv_stats = get_mae_mse_r2_df(
    X_train_poly_ridge, y_train, X_test_poly_ridge, y_test, ridge_cv_model, 'Ridge-CV Model')

display(ridge_cv_stats)
print(f'Alpha used: {ridge_cv_model.alpha_}')


Again, our results did not improve, and our model chose the smallest $\alpha = 1*10^{-10}$, which means that we are (almost) not regularizing at all, and our model objective is equivalent to ordinary least squares demonstrated by our results in the `Multivariate LinearRegression`... So we might need to encode our categorical data better.


In [ ]:

# Using the previous created function to display in nicer format
display(poly_and_ridge_comparison_df)


### Question 5: One-Hot encoding

We will check how the encoding influenced our results.

- Split your original dataset between training and test set (using the same parameters as in Question 2). This time, encode the column 'Item' using `OneHotEncoder`. Finally, rescale your features. **1 point**


In [ ]:
# Creating function to do One-Hot encoding, takes a dataframe and the column to encode
# Returns the new dataframe

def get_ohe(df, column) -> pd.DataFrame:
    '''
    Function that takes a dataframe and one-hot encodes the given column
    '''
    ohe = OneHotEncoder()  # Instantiating the OneHotEncoder (ohe)
    ohe.fit(df[[column]])  # Fitting the ohe on the given column

    # Transforming the given column into a one-hot encoded matrix
    ohm = ohe.transform(df[[column]]).toarray()
    one_hot_df = pd.DataFrame(  # Creating a new DataFrame with the one-hot encoded matrix and column names
        ohm, columns=ohe.get_feature_names_out([column]))

    one_hot_df = one_hot_df.reset_index(drop=True)
    # Concatenate the one-hot encoded DataFrame with the original DataFrame
    df_ret = pd.concat([df.reset_index(drop=True), one_hot_df], axis=1)

    df_ret.drop(column, axis=1, inplace=True)  # Drop the original column
    return df_ret


# Creating a copy of our X's to keep things clean
X_ohe = X.copy()

# Splitting data before encoding to avoid data leakage
X_train_ohe, X_test_ohe, y_train_ohe, y_test_ohe = train_test_split(
    X_ohe, y, test_size=0.2, random_state=42, shuffle=True)

# Calling the One-Hot encoding function
X_train_ohe = get_ohe(X_train_ohe, 'Item')
X_test_ohe = get_ohe(X_test_ohe, 'Item')


In [ ]:

# Now we can rescale our features using our previously  defined MinMaxScaler function
min_max_scaler = MinMaxScaler
X_train_ohe, X_test_ohe = get_scaler(X_train_ohe, X_test_ohe, min_max_scaler)


- Build, train, and evaluate a polynomial features regression model, with the same number of degrees as before, but this time with the one-hot encoded data. How does your model compares to the polynomial features regression model (Question 3)? **2 points**


Now, with the one-hot encoder we have added a lot of new features to our model, which increases the time complexity of our model. Hence, we either could (1) reduce the number of features, by removing those with little correlation, (2) reduce the number of polynomial degrees or (3) reduce the sample size. I will try to discover a combination of these strategies. So the number of features will now be:
$$\text{Polynomial Degree} = 6$$
$$ k =\begin{pmatrix}6+13\\6\end{pmatrix} = \frac{(19)!}{(6! * 13!)} = 27132$$
which is more than 100 times larger than our previous model with 220 features. So the first thing i tried was to use the `RFE` which reduces the size of the features by excluding the least important features. I will build the degrees up and see how much is tolerated in regards to time spent...


In [ ]:

# Perform feature selection on training data using RFE
model = LinearRegression()

# Showcasing poly degree 5 and 50 features
rfe = RFE(model, n_features_to_select=50)
X_train_reduced = rfe.fit_transform(X_train_ohe, y_train_ohe)
ohe_poly_selected_df = get_poly_stats(5, X_train_reduced,  y_train_ohe, rfe.transform(X_test_ohe), y_test_ohe, additonal_title='- OHE (with 50 selected features)')
ohe_poly_selected_df



I found that keeping 50 features and using 5 degree ran in 1.75 minutes and gave an $R^2=0.84$ which is quite a lot better than our polynomial feature model with the same amount of degrees which gave: $PolyModel(5)=>R^2=0,47$. I tried playing around with degree 6 on the one-hot encoded data, but i had to reduce the features to the point where it performed poorer than the 5-degree model.

Another strategy would be to reduce the training set and see if we could keep the polynomial degree. Intuitively this would not reduce the running time that much, since it's the total amount of features that really punishes the running time. But let's try to see if we can do degree 6 by reducing both the set we built our model on and reduce the number of features. I tried by reducing the training set by 50% and only keep the top 10 features. It should run in a little over a minute...

In [ ]:

# Reducing the sets
X_train_small, X_test_small, y_train_small, y_test_small = train_test_split(X_ohe, y, train_size=0.5, random_state=42)
X_train_ohe_small, X_test_ohe_small, y_train_ohe_small, y_test_ohe_small = train_test_split(X_train_small, y_train_small, test_size=0.2, random_state=42, shuffle=True)

# Calling the One-Hot encoding function
X_train_ohe_small = get_ohe(X_train_ohe_small, 'Item')
X_test_ohe_small = get_ohe(X_test_ohe_small, 'Item')

# Now we can rescale our features using our previously  defined get_scaler function
min_max_scaler = MinMaxScaler
X_train_ohe_small, X_test_ohe_small = get_scaler(X_train_ohe_small, X_test_ohe_small, min_max_scaler)

# Reducing the amount of features
model = LinearRegression()
rfe = RFE(model, n_features_to_select=10)
X_train_ohe_small = rfe.fit_transform(X_train_ohe_small, y_train_ohe_small)

# Testing model and getting results
ohe_poly_double_reduced_df = get_poly_stats(6, X_train_ohe_small,  y_train_ohe_small, rfe.transform(X_test_ohe_small), y_test_ohe_small, additonal_title=' (50 selected features and reduced training set)')
ohe_poly_double_reduced_df


This should run in under a minute, and it gives an $R^2=0.77$. Reducing the data that the model get to train on seems not to work very well. So to sum up, lets compare the best scoring model here with the one we obtained in Question 3.

In [ ]:

encoder_comparison_df = pd.concat([poly_and_ridge_comparison_df.iloc[:,4], ohe_poly_selected_df], axis=1)
display(encoder_comparison_df)


We see that the one-hot encoder with 50 of the "best" features on polynomial degree 5 outperforms the one obtained in Q3. So - the `OneHotEncoder` performs much better than the `LabelEncoder`, and the Ridge and RidgeCV model with the LabelEncoded values of `Item`.

## Classification

Access to safe drinking-water is essential to health, a basic human right and a component of effective policy for health protection. However, for a least 3 billion people, the quality of the water they depend on is unknown due to a lack of monitoring (see [SDG Goal 6](https://sdgs.un.org/goals/goal6) 'Ensure availability and sustainable management of water and sanitation for all').

<img src='https://upload.wikimedia.org/wikipedia/commons/thumb/8/87/Sustainable_Development_Goal_6.png/800px-Sustainable_Development_Goal_6.png' width='200'>

We will use data from the [Water Quality](https://www.kaggle.com/datasets/mssmartypants/water-quality) dataset to try to predict whether the water is safe to drink depending on the concentration of various minerals and microorganisms. Check the webpage to read a description of the features and get a better understanding of our problem.


### Question 6: Load and Discover the dataset


- Load the data in a dataframe. The url link is provided below. Display the first 10 observations and the types of data


In [ ]:
url_water = 'https://raw.githubusercontent.com/michalis0/MGT-502-Data-Science-and-Machine-Learning/main/data/waterQuality1.csv'
water_df = pd.read_csv(url_water)

display(water_df.head(10))
display(water_df.dtypes)


- Display summary statistics of your dataset and a heatmap of your correlation matrix **1 point**


In [ ]:
display(water_df.describe())
heat_map_q6 = get_heatmap(water_df.corr(), annot_size=30, title='Correlation Matrix')


- Create a pairplot including the columns 'arsenic', 'cadmium', 'chromium', 'copper', 'bacteria', 'viruses', 'lead', 'nitrates', 'mercury'; and color by the class 'is_safe' **1 points**


In [ ]:

# Features and outcome 
cols_classification = ['arsenic', 'cadmium', 'chromium', 'copper',
                       'bacteria', 'viruses', 'lead', 'nitrates', 'mercury', 'is_safe']

# Getting subset
df_water_subset = get_df_subset(water_df, cols_classification)

# Creating pairplot
water_pair_plot = plot_pairplot(
    data=water_df, title='Water Pairplot', hue='is_safe', columns=cols_classification)


- Feel free to pursue your exploration to better understand your dataset. Although not graded, this might help you better understanding the problem and answer the following questions.


In [ ]:
display(water_df[water_df < 0].count())


I assume that we cannot have negative values for ammonia, so i will remove these 10 rows. I noticed this at the end, and it changed my results a bit for Question 7-10. All the other features are above or equal to 0.

In [ ]:

water_df.drop(water_df[water_df['ammonia'] < 0].index, inplace=True)

### Question 7: Preprocessing


We will try to predict the class 'is_safe', using all the other features.


- Extract your features and outcome. How many observations do we have of class 0 and of class 1? **1 point**


I assume by saying all the other features, it's meant that we shall use all columns except 'is_safe'.

In [ ]:

# Outcome label
outcome_q7 = 'is_safe'

# All features without is_safe
feature_df = water_df.drop(columns={'is_safe'}).copy()
features_classification = feature_df.columns.tolist()

# X and y
X_class = feature_df
y_class = water_df[outcome_q7]

feature_df


In [ ]:
y_class.value_counts().plot.bar(color=['purple', 'blue'], grid=False)
plt.ylabel('Number of observations')
plt.title('Number of observations of each class in the wine dataset')
plt.show()

print(
    f'Unsafe: {y_class.value_counts()[0]}, Safe: {y_class.value_counts()[1]}')


We have 7076 unsafe (class 0) and 910 safe (class 1) observations. We see that we have a heavy imbalance in our data, but since we care more about the characteristics of unsafe water, this might be alright.

- Split between training and test set **1 point**

_Note_: Use as parameters for splitting: `test_size=0.2`, `random_state=39`, `shuffle=True`


In [ ]:

X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(
    X_class, y_class, test_size=0.2, random_state=39, shuffle=True)


- Rescale your features using `StandardScaler` **1 point**


In [ ]:

# Defining the standard scaler
standard_scaler = StandardScaler

# Using the previous created function to scale our data
X_train_class, X_test_class = get_scaler(
    X_train_class, X_test_class, standard_scaler)


### Question 8: Logistic Regression


- Build and train a logistic regression classifier, using as parameters `penalty='l2'`, `solver='lbfgs'`, `max_iter=1000` **1 point**


Just to clarify and note down what we are trying to do, we are using the Sigmoid function to predict our outcome : water safety.

This prediction can be defined by the following equation:
\begin{align*}
WaterSafety[0|1] = &\ w_0 + w_{i/0} \times[features] + \epsilon_i
\end{align*}


And using the Sigmoid function we bound our result between $[0,1]$:
$$S(WaterSafety) = \frac{1}{1 + e^{-WaterSafety}}$$


In [ ]:

# Setting up logistic model
logistic_model = LogisticRegression(
    penalty='l2', solver='lbfgs', max_iter=1000)

# Fitting model
logistic_model.fit(X_train_class, y_train_class)


- Compute the accuracy on the training and test set. Compare it to the default rate. **1 point**


In [ ]:
# Function to create a pretty table

def get_pretty_table(names, rows):
    '''
    Function that return a prettytable, with the given name for titles and rows
    '''
    table = pt.PrettyTable()
    table.field_names = names

    for row in rows:
        table.add_row(row)

    # Add borders to the table
    table.hrules = pt.ALL
    table.header = True
    table.set_style(pt.SINGLE_BORDER)

    return table


In [ ]:

# Accuracy on the test set
train_score = logistic_model.score(X_train_class, y_train_class)
test_score = logistic_model.score(X_test_class, y_test_class)

# Calculating default rate
quality_0 = water_df.loc[water_df[outcome_q7] == 0].shape[0]
quality_1 = water_df.loc[water_df[outcome_q7] == 1].shape[0]
default_rate = max(quality_0, quality_1)/water_df.shape[0]

# Using the get_pretty_table function to print in a nice way
print(get_pretty_table(['Logistic Regression Metrics', 'Values'],
                       [['Accuracy of Logistic regression classifier (training)', round(train_score, 3)],
                       ['Accuracy of Logistic regression classifier (test)', round(test_score, 3)],
                        ['Default rate', round(default_rate, 3)]
                        ]))


We can see that we are a little above the default rate. We perform a bit better than guessing unsafe all the time.

- Plot a heatmap of the confusion matrix. Class 1 is the positive class. How many false positive did we obtain? **1 point**


In [ ]:

# Predicting based on our trained logistic model
pred_class = logistic_model.predict(X_test_class)

# Use previous function to get heatmap
get_heatmap(confusion_matrix(y_test_class, pred_class), title='Confusion Matrix Logistic Regression', color='Blues', vmin=confusion_matrix(
    y_test_class, pred_class).min(), vmax=confusion_matrix(y_test_class, pred_class).max())
plt.xlabel('Predicted label')
plt.ylabel('True labels')
plt.show()

tn, fp, fn, tp = confusion_matrix(y_test_class, pred_class).ravel()
print(f'False positives: {fp}')

We got 19 false positives.

- What is the precision, recall, and f1 score of class 1? Interpret the result. **1 point**


In [ ]:
def get_prec_rec_f1(pred, y_test) -> tuple[float]:
    '''
    Function that returns the Precision, Recall and F1-score
    of the given predicted y-values values and  actual y values with the given model
    '''
    prec = precision_score(y_test, pred, zero_division=1)
    rec = recall_score(y_test, pred, zero_division=1)
    f1 = f1_score(y_test, pred, zero_division=1)
    return prec, rec, f1


def get_prec_rec_f1_df(X_train, X_test, y_train, y_test, model, column_name) -> pd.DataFrame:
    '''
    Function that returns a dataframe with the key performance features on training and test set
    '''
    # Predicting based on given model. Makes the function more dynamic
    pred = model.predict(X_test_class)

    # Accuracy on the test set
    train_score = model.score(X_train, y_train)
    test_score = model.score(X_test, y_test)

    # Getting performance
    prec, rec, f1 = get_prec_rec_f1(
        pred=pred, y_test=y_test)

    # Setting generic labels
    perf_tests_labels = ['Accuracy Training',
                         'Accuracy Test',
                         'Precision (Class 1)',
                         'Recall (Class 1)',
                         'F1-score (Class 1)']

    # Putting together dataframe
    df = pd.DataFrame([train_score, test_score, prec, rec, f1],
                      index=perf_tests_labels,
                      columns=[column_name])

    df = df.apply(lambda x: round(x, 3))
    return df

# Now we can use the function created to get all the performance metrics we want
perf_logistic_df = get_prec_rec_f1_df(X_train_class, X_test_class, y_train_class,y_test_class, logistic_model, 'Logistic Model Performance')
display(perf_logistic_df)


Reminder that;
$$Precision = \frac{TP}{TP+FP}, \quad Recall=\frac{TP}{TP+FN}, \quad and \quad F1 = 2 \frac{{Precision}\cdot{Recall}}{Precision+Recall}$$

With a precision at ~$70$% we predicted correctly when we predicted Class 1 ~70% of the time. Meaning the fraction of correct predicted positives over total positive prediction.  With a recall of $30$%, we are only correctly predicting the safe water ~$30$% of the time. Which means that out of the actual safe water, we predict safe water to be unsafe ~$70$% of the time. The $\text{F1 Score}$ is bounded in the interval $[0,1]$, where 1 means perfect precision and recall, which is low and indicates that the model is not performing well on the positive class. Which we can argue is ok, since it's unsafe water quality that is the most important thing to be good at. 

- Build and train a logistic regression classifier with cross-validation, using 5 folds **1 point**


In [ ]:


logistic_model_cv = LogisticRegressionCV(penalty='l2', solver='lbfgs', cv=5, max_iter=1000)
logistic_model_cv.fit(X_train_class, y_train_class)



- Plot a heatmap of the confusion matrix. Compute the accuracy on the training and test set; as well as the precision, recall, and f1 score of class 1. How do your metrics compare to your model without cross-validation? **1 point**

_Note_: You can, but not have to, create a function to calculate your evaluation metrics since we will perform the same operation later on.


In [ ]:
# Predicting based on our trained logistic CV model
pred_class_cv = logistic_model_cv.predict(X_test_class)

get_heatmap(confusion_matrix(y_test_class, pred_class_cv), title='Confusion Matrix CV Model', color='Blues', vmin=confusion_matrix(
    y_test_class, pred_class_cv).min(), vmax=confusion_matrix(y_test_class, pred_class_cv).max())
plt.xlabel('Predicted label')
plt.ylabel('True labels')
plt.show()


In [ ]:
# Again, we can re-use the function previously created
perf_cv_df = get_prec_rec_f1_df(X_train_class, X_test_class, y_train_class,
                                y_test_class, logistic_model_cv, 'Logistic CV Model Performance')

perf_comparison_df = perf_logistic_df.copy()
perf_comparison_df[perf_cv_df.columns[0]] = perf_cv_df
perf_comparison_df


We can see that the CV model does not perform better than the normal logistic model. The same happens here. To not repeat myself by this could be due to the imbalanced nature of the dataset, where Class 1 has very few samples compared to Class 0. Which is exactly what we saw earlier. The training-set only includes $0.2 * 91+ =  182$ "good water-quality" observations, making the model bad a predicting good water. Thus the model could be biased towards predicting Class 0 (unsafe) more often. Additionally, we can see from the correlation matrix heatmap that each individual feature does not have a huge correlation with the water safety. This could also impact our results. And as mentioned, the precision is more important than recall. But again, when it comes to drinking water it's better to be safe than sorry, even tough a more balanced dataset would make it easier to distinguish the two categories. Moreover, CV might not give us any better performance since we have a total of 8000 observations and the shuffle makes the sets we use to train our model representative, hence CV does not help. Even more, if the relationship is non-linear, logistic regression is not the right algorithm to solve this problem.

### Question 9: KNN classifier


- Build and train a KNN classifier with parameters `n_neighbors=7`, `p=2`, `weights='uniform'` **1 point**


In [ ]:


# Copying train and test set from previous exercise
X_train_knn, X_test_knn, y_train_knn, y_test_knn = X_train_class, X_test_class, y_train_class, y_test_class

# Setting up the model, weights='uniform' is default
model_kNN = KNeighborsClassifier(n_neighbors=7, p=2)

# Fit our model
model_kNN.fit(X_train_knn, y_train_knn)

- Plot a heatmap of the confusion matrix. Compute the accuracy on the training and test set; as well as the precision, recall, and f1 score of class 1. How do your metrics compare to your previous models? **1 point**


In [ ]:
# Predicting based on our trained KNN model
pred_knn = model_kNN.predict(X_test_knn)

get_heatmap(confusion_matrix(y_test_knn, pred_knn), title='Confusion Matrix KNN Model', color='Blues', vmin=confusion_matrix(
    y_test_knn, pred_knn).min(), vmax=confusion_matrix(y_test_knn, pred_knn).max())
plt.xlabel('Predicted label')
plt.ylabel('True labels')
plt.show()

# Getting performance metrics
perf_knn_df = get_prec_rec_f1_df(X_train_knn, X_test_knn, y_train_knn,y_test_knn, model_kNN, 'KNN Model Performance')

perf_knn_comparison_df = perf_comparison_df.copy()
perf_knn_comparison_df[perf_knn_df.columns[0]] = perf_knn_df
display(perf_knn_comparison_df)

Conclusion **before removing the negative values for ammonia**: We clearly see here that the KNN model performs poorly. This could be due to the fact that the KNN model is not sophisticated enough to capture the relationship between our features and the outcome. It might also bee that the KNN model handles the imbalanced data worse than our logistic models. So it could suggest that the decision boundary between the two classes is not highly dependent on the distance/proximity of the nearest neighbors in the feature-dimensional space...

**New conclusion** the KNN model performs a bit better than our Logistic Model. It was cool to see that removing 10 negative variables could have such a huge impact. Since we are using the scaler, our values goes from [-1,1] so it's not the fact that they where negative, but they where probably wrong. The nearest neighbor with euclidean distance was really punished by these wrong datapoints. The KNN algorithm could be better as well due to the fact that in handles non-linearity better than logistic regression.  (Previously i had (negative ammonia) a precision and recall around 0.4). Note to self: Check your data!


- Use `GridSearchCV` to explore different parameters for your model: `n_neighbors` between 1 and 11, `p` between 1 and 3, and `weights` either 'uniform' or 'distance' **1 point**


I assume that when it's written between:

- "Between 1 and 11" = [1,10] = [1,2,3,4,5,6,8,9,10]
- "Between 1 and 3" = [1,2] = [1,2]


In [ ]:

# Defining ranges
neighbors = np.arange(1, 11)
p = np.arange(1, 3)

grid = {'n_neighbors': neighbors,
        'p': p,
        'weights': ['uniform', 'distance']   # weights
        }

# Defining and fit model with optimal
knn = KNeighborsClassifier()
knn_cv = GridSearchCV(knn, grid, cv=10)
knn_cv.fit(X_train_knn, y_train_knn)

# Printing results
print("Hyperparameters: ", knn_cv.best_params_)
print("Train Score: {:0.2f}".format(knn_cv.best_score_))
print("Test Score: {:0.2f}".format(knn_cv.score(X_test_knn, y_test_knn)))


- For your 'optimal' model, compute the accuracy on the training and test set; as well as the precision, recall, and f1 score of class 1. How do your metrics compare to your previous models? **1 point**


In [ ]:
# Set up our model
model_kNN_optimal = KNeighborsClassifier(n_neighbors=9, p=2)

# Fit our model
model_kNN_optimal.fit(X_train_knn, y_train_knn)

perf_knn_df = get_prec_rec_f1_df(X_train_knn, X_test_knn, y_train_knn,
                                 y_test_knn, model_kNN_optimal, 'Optimal KNN Model Performance')


perf_knn_comparison_df[perf_knn_df.columns[0]] = perf_knn_df
display(perf_knn_comparison_df)


We can see that the model does not really change. Maybe a slight improvement in Precision. Still performing better than the logistic models.

### Question 10: Decision Trees


- Build and train a Decision Tree with parameters `criterion = 'gini'`, `max_depth = 3` **1 point**


In [ ]:
from sklearn.tree import DecisionTreeClassifier


# Now with decision tree we don't need the scaled data, DT can even handle categorical data
X_train_tree, X_test_tree, y_train_tree, y_test_tree = train_test_split(
    X_class, y_class, test_size=0.2, random_state=39, shuffle=True)

# Copying train and test set from previous exercise to keep things clean
X_train_tree, X_test_tree, y_train_tree, y_test_tree = X_train_class, X_test_class, y_train_class, y_test_class



# Create model
model_tree = DecisionTreeClassifier(criterion='gini', max_depth=3)

# Fit model
model_tree.fit(X_train_tree, y_train_tree)

I tested the three with both scaled and not scaled data, the latter had 1% better recall for the optimal solution. Otherwise they where almost identical. 

- Plot a heatmap of the confusion matrix. Compute the accuracy on the training and test set; as well as the precision, recall, and f1 score of class 1. How do your metrics compare to your previous models? **1 point**


In [ ]:

# Predicting based on our trained Decision Tree
pred_tree = model_tree.predict(X_test_tree)

get_heatmap(confusion_matrix(y_test_knn, pred_tree), title='Confusion Matrix Decision Tree', color='Blues', vmin=confusion_matrix(
    y_test_tree, pred_tree).min(), vmax=confusion_matrix(y_test_tree, pred_tree).max())
plt.xlabel('Predicted label')
plt.ylabel('True labels')
plt.show()

# Getting performance metrics
perf_tree_df = get_prec_rec_f1_df(
    X_train_tree, X_test_tree, y_train_tree, y_test_tree, model_tree, 'Decision Tree Performance')

all_comparison = perf_knn_comparison_df.copy()

all_comparison[perf_tree_df.columns[0]] = perf_tree_df
display(all_comparison)


The decision tree performs even better than the previous models! First model with a precision over 90%. We have a quite large set of features, which could favour the decision tree over KNN and Logistic regression. The Decision tree can capture nonlinear relationships between the features and the target variable. On the other side, logistic regression assumes that this relationship is linear.

- Visualize your Decision Tree **1 point**


In [ ]:
# Creating figure
plt.figure(figsize=(15, 10))
plot_tree(model_tree, filled=True,
          feature_names=features_classification, fontsize=12, rounded=True)

# Setting some labels
plt.title("Decision Tree Classifier", fontsize=20)
plt.xlabel("Features", fontsize=16)
plt.ylabel("Target Variable", fontsize=16)
plt.show()


- Use `GridSearchCV` to explore different parameters for your model: `criterion` either 'gini' or 'entropy' and `max_depth` between 1 and 7 **1 point**


In [ ]:
# Define parameters to test
grid_tree = {'criterion': ['gini', 'entropy'], 'max_depth': np.arange(1, 8)}

# Define and fit model
tree_class = DecisionTreeClassifier()
tree_class_cv = GridSearchCV(tree_class, grid_tree, cv=5)
tree_class_cv.fit(X_train_tree, y_train_tree)

# Print results
print("Best model:", tree_class_cv.best_estimator_)
print("Train Score: {:0.2f}".format(tree_class_cv.best_score_))
print("Test Score: {:0.2f}".format(tree_class_cv.score(X_test_tree, y_test_tree)))


- For your 'optimal' model, compute the accuracy on the training and test set; as well as the precision, recall, and f1 score of class 1. How do your metrics compare to your previous models? **1 point**


In [ ]:


model_tree_optimal = DecisionTreeClassifier(criterion='entropy', max_depth=7)

# Fit model
model_tree_optimal.fit(X_train_tree, y_train_tree)

# Predicting based on our trained Decision Tree
pred_tree_optimal = model_tree_optimal.predict(X_test_tree)

get_heatmap(confusion_matrix(y_test_knn, pred_tree_optimal), title='Confusion Matrix Optimal Decision Tree', color='Blues', vmin=confusion_matrix(
    y_test_tree, pred_tree).min(), vmax=confusion_matrix(y_test_tree, pred_tree_optimal).max())
plt.xlabel('Predicted label')
plt.ylabel('True labels')
plt.show()

# Getting performance metrics
perf_tree_optimal_df = get_prec_rec_f1_df(
    X_train_tree, X_test_tree, y_train_tree, y_test_tree, model_tree_optimal, 'Optimal Decision Tree Performance')


tn, fp, fn, tp = confusion_matrix(y_test_knn, pred_tree_optimal).ravel()

print(f'\nTrue Negatives: {tn}\
        \nFalse Negatives: {fn}\
        \nFalse Positives: {fp}\
        True Positives: {tp}')

all_comparison[perf_tree_optimal_df.columns[0]] = perf_tree_optimal_df
display(all_comparison)


Best accuracy on training set so far, and almost the same precision as the previous decision tree. But not the lowest false positive which might be what we really care about in this case which is related to how many times have we told people that the water is safe, but it's actually not. Overall the decisions tree performed well! Well suited for a lot of features with a complex relation to the outcome variable.

Congrats, you are done with the assignment!


In [ ]:
%load_ext watermark
%watermark -v -p pandas,numpy,sklearn